## L-mode H-mode Classifier
- In this notebook a single camera model will be created and trained. 
- First import ResNet model, modify its fc. layer and train this fc.layer
- Then train the whole model
- We can either train the model on imgs from RIS1 camera or RIS2, and then ensemble these models in `ModelEnsembling.ipynb`

In [2]:
import os
import re
import time 
from pathlib import Path
from datetime import datetime

import matplotlib.pyplot as plt
from torch import cuda
import torchvision
import torch
from torch.optim import lr_scheduler
from tqdm import tqdm
import torch.nn as nn
from torch.utils.tensorboard import SummaryWriter
import pandas as pd
import pytorch_lightning as pl

import confinement_mode_classifier as cmc

pl.seed_everything(42)

Seed set to 42


42

In [3]:
path = Path(os.getcwd())
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")

#Where are csv spreadsheets with time, mode, img_path, h_alpha columns are saved
data_dir_path = f'{path}/data/LH_alpha'
file_names = os.listdir(data_dir_path)


## Constants

In [6]:
ris_option = 'RIS1'
num_workers = 2
num_epochs_for_fc = 10
num_epochs_for_all_layers = 10
num_classes = 2
batch_size = 32
learning_rate_min = 0.001
learning_rate_max = 0.01
comment_for_model_name = ris_option + f'2 output classes'

### Create dataloader
- It is convinient to have dloaders in dictionary in order to easily switch between training and validation

In [7]:
shot_usage = pd.read_csv(f'{path}/data/shot_usage.csv')
shot_for_ris = shot_usage[shot_usage['used_for_ris1'] if ris_option == 'RIS1' else shot_usage['used_for_ris2']]
shot_numbers = shot_for_ris['shot']
shots_for_testing = shot_for_ris[shot_for_ris['used_as'] == 'test']['shot']
shots_for_validation = shot_for_ris[shot_for_ris['used_as'] == 'val']['shot']

shot_df, test_df, val_df, train_df = cmc.load_and_split_dataframes(path,shot_numbers, shots_for_testing, 
                                                                   shots_for_validation, use_ELMS=False)


test_dataloader = cmc.get_dloader(test_df, path, batch_size, 
                                    ris_option=ris_option, balance_data=False, 
                                    shuffle=False, num_workers=num_workers)

val_dataloader = cmc.get_dloader(val_df, path, batch_size, 
                                    ris_option=ris_option, balance_data=True, 
                                    shuffle=False, num_workers=num_workers)

train_dataloader = cmc.get_dloader(train_df, path, batch_size, 
                                    ris_option=ris_option, balance_data=True, 
                                    shuffle=False, num_workers=num_workers)

dataloaders = {'train':train_dataloader, 'val':val_dataloader}
dataset_sizes = {x: len(dataloaders[x].dataset) for x in ['train', 'val']}

Time stamp will be added to the model's folder name

In [9]:
train_df[train_df['shot'] == 18128]

,time,mode,filename,h_alpha,shot


In [5]:
timestamp =  datetime.fromtimestamp(time.time()).strftime("%y-%m-%d, %H-%M-%S ") + comment_for_model_name
writer = SummaryWriter(f'runs/{timestamp}_last_fc')

### Import ResNet pretrained model
 And freeze all layers except last f.c. layer


In [8]:
pretrained_model = torchvision.models.resnet18(weights='IMAGENET1K_V1')
for param in pretrained_model.parameters():
    param.requires_grad = False
 
# Parameters of newly constructed modules have requires_grad=True by default
num_ftrs = pretrained_model.fc.in_features
pretrained_model.fc = nn.Linear(num_ftrs, num_classes) #3 classes: L-mode, H-mode, ELM
pretrained_model = pretrained_model.to(device)

In [17]:
sample_input = next(iter(train_dataloader))['img']
writer.add_graph(pretrained_model, sample_input.float().to(device))


### Train last fc of the model

In [ ]:
#
criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer = torch.optim.Adam(pretrained_model.parameters(), lr=learning_rate_min)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.OneCycleLR(optimizer, max_lr=learning_rate_max, total_steps=50) #!!!

#Model will be saved to this folder along with metrics and tensorboard scalars
model_path = Path(f'{path}/runs/{timestamp}_last_fc/model.pt')


model = cmc.train_model(pretrained_model, criterion, optimizer, exp_lr_scheduler, 
                       dataloaders, writer, dataset_sizes, num_epochs=num_epochs_for_fc, 
                       chkpt_path=model_path.with_name(f'{model_path.stem}_chkpt{model_path.suffix}'))

torch.save(model.state_dict(), model_path)

### Test model

In [ ]:
metrics = cmc.test_model(f'runs/{timestamp}', model, test_dataloader, comment='all_layers')

shots_for_testing = shots_for_testing.values.tolist()
img_path = cmc.per_shot_test(path=f'{path}/runs/{timestamp}_last_fc/', 
                             shots=shots_for_testing, results_df=metrics['prediction_df'], writer=writer)

writer.add_figure(f'Confusion matrix for the model with trained f.c. layer', metrics['confusion_matrix'][0])
writer.add_figure(f'Confusion matrix for the model with trained f.c. layer', metrics['confusion_matrix'][0])
writer.add_scalar(f'Accuracy on test_dataset with trained f.c. layer', metrics['accuracy'])
writer.add_scalar(f'F1 metric on test_dataset with trained f.c. layer', metrics['f1'])
writer.add_scalar(f'Precision on test_dataset with trained f.c. layer', metrics['precision'])
writer.add_scalar(f'Recall on test_dataset with trained f.c. layer', metrics['recall'])
writer.close()

### Train the whole model

In [ ]:
writer = SummaryWriter(f'runs/{timestamp}_all_layers')
for param in model.parameters():
    param.requires_grad = True

model_path = Path(f'{path}/runs/{timestamp}_all_layers/model.pt')



model = cmc.train_model(model, criterion, optimizer, exp_lr_scheduler, 
                        dataloaders, writer, dataset_sizes, num_epochs=num_epochs_for_all_layers,
                        chkpt_path=model_path.with_name(f'{model_path.stem}_chkpt{model_path.suffix}'))
torch.save(model.state_dict(), model_path)

### Test model with all layers trained

In [ ]:
metrics = cmc.test_model(f'runs/{timestamp}', model, test_dataloader, comment='all_layers')

shots_for_testing = shots_for_testing.values.tolist()
img_path = cmc.per_shot_test(path=f'{path}/runs/{timestamp}_last_fc/', 
                             shots=shots_for_testing, results_df=metrics['prediction_df'], writer=writer)

writer.add_figure(f'Confusion matrix for the model with trained f.c. layer', metrics['confusion_matrix'][0])
writer.add_figure(f'Confusion matrix for the model with trained f.c. layer', metrics['confusion_matrix'][0])
writer.add_scalar(f'Accuracy on test_dataset with trained f.c. layer', metrics['accuracy'])
writer.add_scalar(f'F1 metric on test_dataset with trained f.c. layer', metrics['f1'])
writer.add_scalar(f'Precision on test_dataset with trained f.c. layer', metrics['precision'])
writer.add_scalar(f'Recall on test_dataset with trained f.c. layer', metrics['recall'])
writer.close()